## 14. 서브워드 토크나이저(Subword Tokenizer)
* 단어 토큰화에서 더 나아가 단어를 서브워드 단위까지 나누는 서브워드 토큰화를 하는 이유
* 서브워드 토큰화를 수행하는 서브워드 토크나이저의 알고리즘과 그 구현체

### 01) 바이트페어 인코딩(Byte Pair Encoding, BPE)
* 기계가 모르는 단어: OOV(Out of Vocabulary) 또는 UNK(Unknown Token)
* **OOV 문제**: 모르는 단어로 인해 문제를 푸는 것이 까다로워지는 상황



* **서브워드 분리(Subword segmentation)**: 하나의 단어는 더 작은 단위의 의미있는 여러 서브워드들의 조합으로 구성된 경우가 많기 때문에, 하나의 단어를 여러 서브워드로 분리해서 단어를 인코딩 및 임베딩하겠다는 의도를 가진 전처리 작업
  * OOV, 희귀 단어, 신조어와 같은 문제를 완화 가능
* **BPE(Byte Pair Encoding)**: OOV 문제를 완화하는 대표적인 서브워드 분리 알고리즘
* **Sentencepiece**: 이 알고리즘을 실무에서 사용할 수 있도록 구현

#### 1. BPE
* 글자 대신 byte라는 표현 사용
#### 2. 자연어처리에서의 BPE
* 자연어 처리에서의 BPE는 서브워드 분리(Subword segmentation) 알고리즘
* 글자(character) 단위에서 점차적으로 단어 집합(vocabulary)를 만들어 내는 Bottom up 방식의 접근
* 훈련 데이터에 있는 단어들을 모든 글자(characters) 또는 유니코드(unicode) 단위로 단어 집합(vocabulary)를 만들고, 가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합

##### 1) 기존의 접근
* 학습하지 않은 단어가 나오면 해당 단어에 대해서 제대로 대응하지 못하는 OOV 문제가 발생

##### 2) BPE 알고리즘을 사용한 경우
* 우선 딕셔너리의 모든 단어들을 글자(character) 단위로 분리
  * 초기 구성은 글자 단위로 분리된 상태
* 가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정을 총 n번 반복
* BPE에서 'lowest'가 등장하면...
  1. 기계는 우선 lowest를 전부 글자 단위로 분할
  2. 위의 단어 집합을 참고로 하여 low와 est를 찾아냄. 즉, lowest를 기계는 low와 est 두 단어로 인코딩함


##### 3) 코드 실습하기

In [1]:
import re, collections
from IPython.display import display, Markdown, Latex

In [2]:
num_merges = 10 #BPE를 몇 회 수행할 것인지

In [3]:
# low, lower, newest, widest일 때, 
# BPE의 입력으로 사용하는 실제 단어 집합은 아래와 같음

dictionary = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>':6,
         'w i d e s t </w>':3
         }

In [6]:
# 가장 빈도수가 높은 유니그램의 쌍을
# 하나의 유니그램으로 통합하는 과정을 num_merges회 반복

def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    print('현재 pair들의 빈도수 :', dict(pairs))
    return pairs

def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

bpe_codes = {}
bpe_codes_reverse = {}

for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i + 1)))
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print("new merge: {}".format(best))
    print("dictionary: {}".format(dictionary))

### Iteration 1

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
new merge: ('e', 's')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
new merge: ('es', 't')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

현재 pair들의 빈도수 : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('new', 'est</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('new', 'est</w>')
dictionary: {'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 9

현재 pair들의 빈도수 : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('low', '</w>')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

현재 pair들의 빈도수 : {('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('w', 'i')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


In [7]:
print(bpe_codes) #merge했던 기록이 출력

{('e', 's'): 0, ('es', 't'): 1, ('est', '</w>'): 2, ('l', 'o'): 3, ('lo', 'w'): 4, ('n', 'e'): 5, ('ne', 'w'): 6, ('new', 'est</w>'): 7, ('low', '</w>'): 8, ('w', 'i'): 9}


##### 4) OOV에 대처하기

In [8]:
def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as a tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs


def encode(orig):
    """Encode word based on list of BPE merge operations, which are applied consecutively"""

    word = tuple(orig) + ('</w>',)
    display(Markdown("__word split into characters:__ <tt>{}</tt>".format(word)))

    pairs = get_pairs(word)    

    if not pairs:
        return orig

    iteration = 0
    while True:
        iteration += 1
        display(Markdown("__Iteration {}:__".format(iteration)))

        print("bigrams in the word: {}".format(pairs))
        bigram = min(pairs, key = lambda pair: bpe_codes.get(pair, float('inf')))
        print("candidate for merging: {}".format(bigram))
        if bigram not in bpe_codes:
            display(Markdown("__Candidate not in BPE merges, algorithm stops.__"))
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word)-1 and word[i+1] == second:
                new_word.append(first+second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        print("word after merging: {}".format(word))
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # 특별 토큰인 </w>는 출력하지 않는다.
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>',''),)

    return word

In [9]:
encode("loki")

__word split into characters:__ <tt>('l', 'o', 'k', 'i', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('i', '</w>'), ('l', 'o'), ('o', 'k'), ('k', 'i')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'k', 'i', '</w>')


__Iteration 2:__

bigrams in the word: {('lo', 'k'), ('i', '</w>'), ('k', 'i')}
candidate for merging: ('lo', 'k')


__Candidate not in BPE merges, algorithm stops.__

('lo', 'k', 'i')

In [10]:
encode("lowest")

__word split into characters:__ <tt>('l', 'o', 'w', 'e', 's', 't', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('e', 's'), ('w', 'e'), ('l', 'o'), ('s', 't'), ('o', 'w'), ('t', '</w>')}
candidate for merging: ('e', 's')
word after merging: ('l', 'o', 'w', 'es', 't', '</w>')


__Iteration 2:__

bigrams in the word: {('es', 't'), ('l', 'o'), ('w', 'es'), ('o', 'w'), ('t', '</w>')}
candidate for merging: ('es', 't')
word after merging: ('l', 'o', 'w', 'est', '</w>')


...

bigrams in the word: {('lo', 'w'), ('w', 'est</w>')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'est</w>')


__Iteration 6:__

bigrams in the word: {('low', 'est</w>')}
candidate for merging: ('low', 'est</w>')


__Candidate not in BPE merges, algorithm stops.__

('low', 'est')

In [11]:
encode("lowing")

__word split into characters:__ <tt>('l', 'o', 'w', 'i', 'n', 'g', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('g', '</w>'), ('i', 'n'), ('l', 'o'), ('o', 'w'), ('n', 'g'), ('w', 'i')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'w', 'i', 'n', 'g', '</w>')


__Iteration 2:__

bigrams in the word: {('g', '</w>'), ('i', 'n'), ('lo', 'w'), ('n', 'g'), ('w', 'i')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'i', 'n', 'g', '</w>')


__Iteration 3:__

bigrams in the word: {('n', 'g'), ('i', 'n'), ('g', '</w>'), ('low', 'i')}
candidate for merging: ('n', 'g')


__Candidate not in BPE merges, algorithm stops.__

('low', 'i', 'n', 'g')

In [12]:
encode("highing")

__word split into characters:__ <tt>('h', 'i', 'g', 'h', 'i', 'n', 'g', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('g', '</w>'), ('i', 'n'), ('i', 'g'), ('h', 'i'), ('n', 'g'), ('g', 'h')}
candidate for merging: ('g', '</w>')


__Candidate not in BPE merges, algorithm stops.__

('h', 'i', 'g', 'h', 'i', 'n', 'g')

#### 3. Wordpiece model
* BPE의 변형 알고리즘
* WPM은 BPE가 **빈도수**에 기반하여 가장 많이 등장한 쌍을 병합하는 것과는 달리, **병합되었을 때 코퍼스의 우도(Likelihood)를 가장 높이는 쌍을 병합**
```
WPM을 수행하기 이전의 문장: Jet makers feud over seat width with big orders at stake
WPM을 수행한 결과(wordpieces): _J et _makers _fe ud _over _seat _width _with _big _orders _at _stake
```
* 모든 단어의 맨 앞에 _를 붙이고, 단어는 subword로 통계에 기반하여 띄어쓰기로 분리
* 기존에 있던 띄어쓰기와 구분자 역할의 띄어쓰기는 어떻게 구별?
  * 이 역할을 수행하는 것이 단어들 앞에 붙은 언더바 _
* WPM이 수행된 결과로부터 다시 수행 전의 결과로 돌리는 방법
  * 현재 있는 모든 띄어쓰기를 전부 제거하고, 언더바를 띄어쓰기로 바꾸면 됨
* BERT 훈련에도 사용됨


#### 4. Unigram Language Model Tokenizer
* 각각의 서브워드들에 대해서 **손실(loss)**를 계산
  * 서브 단어의 손실: 해당 서브워드가 단어 집합에서 **제거**되었을 경우, **코퍼스의 우도(Likelihood)가 감소하는 정도**
* 이렇게 측정된 서브워드들은 손실의 정도로 정렬하여, 최악의 영향을 주는 10-20%의 토큰을 제거
  * 이를 원하는 단어 집합의 크기에 도달할 때까지 반복


### 02) 센텐스피스(SentencePiece)
#### 1. 센텐스피스(SentencePiece)
* 구글의 Sentencepiece: BPE 알고리즘과 Unigram Language Model Tokenizer를 구현한 센텐스피스를 깃허브에 공개
* 내부 단어 분리 알고리즘을 사용하기 위해서, 데이터에 단어 토큰화를 먼저 진행한 상태여야 한다면 이 단어 분리 아록리즘을 모든 언어에 사용하는 것은 쉽지 않음
  * 영어와 달리 한국어는 단어 토큰화부터 쉽지 않음
  * 이런 사전 토큰화 작업(pretokenization) 없이 raw data에 바로 단어 분리 토크나이저를 사용할 수 있다면, 이 토크나이저는 어떤 단어에도 적용할 수 있는 토크나이저가 될 것
    * Sentencepiece는 이 이점을 살려서 구현됨
    * **사전 토큰화 작업 없이 단어 분리 토큰화를 수행하므로 언어에 종속되지 않음**

In [13]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.3 MB/s 


#### 2. IMDB 리뷰 토큰화하기

In [15]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

In [16]:
# IMDB 리뷰 데이터를 다운로드하고, 이를 데이터프레임에 저장
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7fc886045d10>)

In [17]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [18]:
print('리뷰 개수 :',len(train_df))

리뷰 개수 : 50000


In [19]:
# 총 5만개의 샘플 존재
# 데이터프레임을 txt 파일로 저장
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

In [20]:
# 센텐스피스로 단어 집합과 각 단어에 고유한 정수 부여
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

'''
input : 학습시킬 파일
model_prefix : 만들어질 모델 이름
vocab_size : 단어 집합의 크기
model_type : 사용할 모델 (unigram(default), bpe, char, word)
max_sentence_length: 문장의 최대 길이
pad_id, pad_piece: pad token id, 값
unk_id, unk_piece: unknown token id, 값
bos_id, bos_piece: begin of sentence token id, 값
eos_id, eos_piece: end of sequence token id, 값
user_defined_symbols: 사용자 정의 토큰
'''

In [21]:
# vocab 생성이 완료되면 imdb.model, imdb.vocab 파일 두 개가 생성됨
# vocab 파일에서 학습된 서브워드들을 확인 가능
# 단어 집합의 크기를 확인하기 위해 vocab 파일을 데이터프레임에 저장

vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
3824,Sh,-3821
2988,▁genuine,-2985
1143,▁ey,-1140
3732,▁Though,-3729
4755,▁(2,-4752
3028,sequ,-3025
854,▁diffe,-851
2491,▁Gra,-2488
1127,▁comm,-1124
117,▁M,-114


In [22]:
len(vocab_list)

5000

In [23]:
# model 파일을 로드하여 단어 시퀀스를 정수 시퀀스로 바꾸는 인코딩 작업이나 반대로 변환하는 디코딩 작업 가능

sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)

True

In [24]:
'''
encode_as_pieces : 문장을 입력하면 서브 워드 시퀀스로 변환합니다.
encode_as_ids : 문장을 입력하면 정수 시퀀스로 변환합니다.
'''

lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]

for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 623, 4950, 4926, 138, 169, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]



In [25]:
sp.GetPieceSize() #단어 집합의 크기 확인

5000

In [26]:
sp.IdToPiece(430) #정수로부터 맵핑되는 서브 워드로 변환

'▁character'

In [27]:
sp.PieceToId('_character') #서브워드로부터 맵핑되는 정수로 변환

0

In [28]:
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]) #정수 시퀀스로부터 문장으로 변환

'I have waited a long time for someone to film'

In [29]:
sp.DecodePieces(['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']) 
#서브워드 시퀀스로부터 문장으로 변환

'I have waited a long time for someone to film'

In [30]:
print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))
#문장으로부터 인자값에 따라서 정수 시퀀스 또는 서브워드 시퀀스로 변환 가능

['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]


#### 3. 네이버 영화 리뷰 토큰화하기

In [31]:
import pandas as pd
import sentencepiece as spm
import urllib.request
import csv

In [32]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
naver_df = pd.read_table('ratings.txt')
naver_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [33]:
print('리뷰 개수 :',len(naver_df)) 

리뷰 개수 : 200000


In [34]:
print(naver_df.isnull().values.any())

True


In [35]:
naver_df = naver_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [36]:
print('리뷰 개수 :',len(naver_df))

리뷰 개수 : 199992


In [37]:
# 199992개의 샘플을 naver_review.txt 파일에 저장한 후에
# 센텐스피스를 통해 단어 집합을 생성
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [38]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [39]:
# vocab 생성이 완료되면 naver.model, naver.vocab 파일 두 개가 생성됨
# vocab에서 학습된 subwords를 확인 가능

vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,▁그,-6


In [40]:
vocab_list.sample(10)

# Vocab에는 unknown,
# 문장의 시작, 문장의 끝을 의미하는 special token이 0, 1, 2에 사용됨

,0,1
3517,경,-3514
1657,유의,-1654
1581,!!!!!!!!,-1578
1254,♥♥,-1251
1414,▁삼류,-1411
3942,슈,-3939
562,▁독,-559
3635,희,-3632
4707,퀼,-4704
3688,범,-3685


In [41]:
len(vocab_list)

5000

In [42]:
sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)

True

In [43]:
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']
[132, 966, 1296, 2590, 3276]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]



In [44]:
sp.GetPieceSize()

5000

In [45]:
sp.IdToPiece(4)

'영화'

In [46]:
sp.PieceToId('영화')

4

In [47]:
sp.DecodeIds([54, 200, 821, 85])

'진짜 최고의 영화입니다 ᄏᄏ'

In [48]:
sp.DecodePieces(['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ'])

'진짜 최고의 영화입니다 ᄏᄏ'

In [49]:
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]


### 03) SubwordTextEncoder
* 텐서플로우를 통해 사용할 수 있는 서브워드 토크나이저
* BPE와 유사한 알고리즘인 Wordpiece Model을 채택. 패키지로 쉽게 단어들을 서브워드들로 분리 가능
#### 1. IMDB 리뷰 토큰화하기

In [50]:
import tensorflow_datasets as tfds
import urllib.request
import pandas as pd

In [51]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7fc80b40ded0>)

In [52]:
train_df = pd.read_csv('IMDb_Reviews.csv')

In [53]:
# 토큰화를 수행해야 할 데이터
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [57]:
# tfds.features.text.SubwordTextEncoder.build_from_corpus의 인자로 토큰화 할 데이터를 넣어줌
# 서브워드들로 이루어진 단어 집합(Vocabulary)를 생성하고, 각 서브워드에 고유한 정수를 부여

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    train_df['review'], target_vocab_size=2**13)

In [58]:
print(tokenizer.subwords[:100])

['the_', ', ', '. ', 'a_', 'and_', 'of_', 'to_', 's_', 'is_', 'br', 'in_', 'I_', 'that_', 'this_', 'it_', ' /><', ' />', 'was_', 'The_', 't_', 'as_', 'with_', 'for_', '.<', 'on_', 'but_', 'movie_', 'are_', ' (', 'have_', 'his_', 'film_', 'not_', 'be_', 'you_', 'ing_', ' "', 'ed_', 'it', 'd_', 'an_', 'at_', 'by_', 'he_', 'one_', 'who_', 'from_', 'y_', 'or_', 'e_', 'like_', 'all_', '" ', 'they_', 'so_', 'just_', 'has_', ') ', 'about_', 'her_', 'out_', 'This_', 'some_', 'movie', 'ly_', 'film', 'very_', 'more_', 'It_', 'what_', 'would_', 'when_', 'if_', 'good_', 'up_', 'which_', 'their_', 'only_', 'even_', 'my_', 'really_', 'had_', 'can_', 'no_', 'were_', 'see_', '? ', 'she_', 'than_', '! ', 'there_', 'been_', 'get_', 'into_', 'will_', ' - ', 'much_', 'n_', 'because_', 'ing']


In [59]:
print(train_df['review'][20])

Pretty bad PRC cheapie which I rarely bother to watch over again, and it's no wonder -- it's slow and creaky and dull as a butter knife. Mad doctor George Zucco is at it again, turning a dimwitted farmhand in overalls (Glenn Strange) into a wolf-man. Unfortunately, the makeup is virtually non-existent, consisting only of a beard and dimestore fangs for the most part. If it were not for Zucco and Strange's presence, along with the cute Anne Nagel, this would be completely unwatchable. Strange, who would go on to play Frankenstein's monster for Unuiversal in two years, does a Lenny impression from "Of Mice and Men", it seems.<br /><br />*1/2 (of Four)


In [ ]:
# encode()를 통해서 입력한 데이터에 대해서 정수 인코딩을 수행한 결과를 얻을 수 있음
print('Tokenized sample question: {}'.format(tokenizer.encode(train_df['review'][20])))

In [60]:
# 임의로 선택한 짧은 문장에 대해서 정수 인코딩 결과를 확인하고, 이를 다시 역으로 디코딩
# train_df에 존재하는 문장 중 일부를 발췌
sample_string = "It's mind-blowing to me that this film was even made."

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print('정수 인코딩 후의 문장 {}'.format(tokenized_string))

# 이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print('기존 문장 : {}'.format(original_string))

정수 인코딩 후의 문장 [137, 8051, 8, 910, 8057, 2169, 36, 7, 103, 830, 1601, 8044, 14, 32, 18, 79, 681, 8058]
기존 문장 : It's mind-blowing to me taht this film was even made.


In [61]:
print('단어 집합의 크기(Vocab size) :', tokenizer.vocab_size)

단어 집합의 크기(Vocab size) : 8268


In [62]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

137 ----> It
8051 ----> '
8 ----> s 
910 ----> mind
8057 ----> -
2169 ----> blow
36 ----> ing 
7 ----> to 
103 ----> me 
830 ----> ta
1601 ----> ht
8044 ---->  
14 ----> this 
32 ----> film 
18 ----> was 
79 ----> even 
681 ----> made
8058 ----> .


In [63]:
# 기존 예제 문장 중 even이라는 단어에 임의로 xyz라는 3개의 글자 추가
# 현재 토크나이저가 even이라는 단어를 이미 하나의 서브워드로 인식하고 있는 상황에서 
# 나머지 xyz를 어떻게 분리하는지를 확인하기 위함

sample_string = "It's mind-blowing to me that this film was evenxyz made."

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print('정수 인코딩 후의 문장 {}'.format(tokenized_string))

# 이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print('기존 문장 : {}'.format(original_string))

정수 인코딩 후의 문장 [137, 8051, 8, 910, 8057, 2169, 36, 7, 103, 13, 14, 32, 18, 7974, 8132, 8133, 997, 681, 8058]
기존 문장 : It's mind-blowing to me that this film was evenxyz made.


In [64]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

# evenxyz에서 even을 독립적으로 분리하고 xyz는 훈련 데이터에서 하나의 단어로서 등장한 적이 없으므로
# 각각 전부 분리

137 ----> It
8051 ----> '
8 ----> s 
910 ----> mind
8057 ----> -
2169 ----> blow
36 ----> ing 
7 ----> to 
103 ----> me 
13 ----> that 
14 ----> this 
32 ----> film 
18 ----> was 
7974 ----> even
8132 ----> x
8133 ----> y
997 ----> z 
681 ----> made
8058 ----> .


#### 2. 네이버 영화 리뷰 토큰화하기

In [65]:
import tensorflow_datasets as tfds
import urllib.request

In [66]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
train_data = pd.read_table('ratings_train.txt')
print(train_data.isnull().sum())

id          0
document    5
label       0
dtype: int64


In [67]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any())

False


In [68]:
# 인자로 네이버 영화 리뷰 데이터를 넣어서
# 서브워드들로 이루어진 단어 집합(Vocabulary)를 생성하고
# 각 서브워드에 고유한 정수 부여
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    train_data['document'], target_vocab_size=2**13)

In [69]:
print(tokenizer.subwords[:100])

['. ', '..', '영화', '이_', '...', '의_', '는_', '도_', '다', ', ', '을_', '고_', '은_', '가_', '에_', '.. ', '한_', '너무_', '정말_', '를_', '고', '게_', '영화_', '지', '... ', '진짜_', '이', '다_', '요', '만_', '? ', '과_', '나', '가', '서_', '지_', '로_', '으로_', '아', '어', '....', '음', '한', '수_', '와_', '도', '네', '그냥_', '나_', '더_', '왜_', '이런_', '면_', '기', '하고_', '보고_', '하는_', '서', '좀_', '리', '자', '스', '안', '! ', '에서_', '영화를_', '미', 'ㅋㅋ', '네요', '시', '주', '라', '는', '오', '없는_', '에', '해', '사', '!!', '영화는_', '마', '잘_', '수', '영화가_', '만', '본_', '로', '그_', '지만_', '대', '은', '비', '의', '일', '개', '있는_', '없다', '함', '구', '하']


In [70]:
# 21번째 샘플을 인코딩
print('Tokenized sample question: {}'.format(tokenizer.encode(train_data['document'][20])))

Tokenized sample question: [669, 4700, 17, 1749, 8, 96, 131, 1, 48, 2239, 4, 7466, 32, 1274, 2655, 7, 80, 749, 1254]


In [71]:
# 역으로 디코딩
sample_string = train_data['document'][21]

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 {}'.format(tokenized_string))

# 이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장: {}'.format(original_string))

정수 인코딩 후의 문장 [570, 892, 36, 584, 159, 7091, 201]
기존 문장: 보면서 웃지 않는 건 불가능하다


In [72]:
# 임의로 작성한 짧은 문장에 대해서 정수 인코딩 결과 확인하고 역으로 디코딩
# 킄핫핫ㅎ 추가

sample_string = '이 영화 굉장히 재밌다 킄핫핫ㅎ'

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 {}'.format(tokenized_string))

# 이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장: {}'.format(original_string))

정수 인코딩 후의 문장 [4, 23, 1364, 2157, 8235, 8128, 8130, 8235, 8147, 8169, 8235, 8147, 8169, 393]
기존 문장: 이 영화 굉장히 재밌다 킄핫핫ㅎ


In [73]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

4 ----> 이 
23 ----> 영화 
1364 ----> 굉장히 
2157 ----> 재밌다 
8235 ----> �
8128 ----> �
8130 ----> �
8235 ----> �
8147 ----> �
8169 ----> �
8235 ----> �
8147 ----> �
8169 ----> �
393 ----> ㅎ
